---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [63]:
import pandas as pd
import numpy as np
train = pd.read_csv('train.csv', encoding='cp1252',  low_memory=False, ) 
test = pd.read_csv('test.csv')
train = train.dropna(subset=['compliance'])
#train = train[np.isfinite(train['compliance'])]

X_train = train.iloc[:,:-1]
y_train = train['compliance']

print(X_train)

        ticket_id                                     agency_name  \
0           22056  Buildings, Safety Engineering & Env Department   
1           27586  Buildings, Safety Engineering & Env Department   
5           22046  Buildings, Safety Engineering & Env Department   
6           18738  Buildings, Safety Engineering & Env Department   
7           18735  Buildings, Safety Engineering & Env Department   
...           ...                                             ...   
250278     285106                      Department of Public Works   
250287     284650                      Department of Public Works   
250288     285125                      Department of Public Works   
250290     284881                      Department of Public Works   
250293     284333                      Department of Public Works   

            inspector_name                         violator_name  \
0          Sims, Martinzie     INVESTMENT INC., MIDWEST MORTGAGE   
1         Williams, Darrin         

In [ ]:
list(X_train)

In [64]:
X_train.drop([ 'agency_name',
 'inspector_name',
 'violator_name',
 'violation_street_number',
 'violation_street_name',
 'violation_zip_code',
 'mailing_address_str_number',
 'mailing_address_str_name',
 'city',
 'state',
 'zip_code',
 'non_us_str_code',
 'country',
 'ticket_issued_date',
 'hearing_date',
 'violation_code',
 'violation_description',
 'disposition',
 'payment_date',
 'payment_status',
 'collection_status',
 'grafitti_status',
 'compliance_detail',
 'balance_due',
'payment_amount' ], axis=1, inplace=True)
X_train

,ticket_id,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount
0,22056,250.0,20.0,10.0,25.0,0.0,0.0,305.0
1,27586,750.0,20.0,10.0,75.0,0.0,0.0,855.0
5,22046,250.0,20.0,10.0,25.0,0.0,0.0,305.0
6,18738,750.0,20.0,10.0,75.0,0.0,0.0,855.0
7,18735,100.0,20.0,10.0,10.0,0.0,0.0,140.0
...,...,...,...,...,...,...,...,...
250278,285106,200.0,20.0,10.0,20.0,0.0,0.0,250.0
250287,284650,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0
250288,285125,500.0,20.0,10.0,50.0,0.0,0.0,580.0
250290,284881,200.0,20.0,10.0,0.0,0.0,0.0,230.0


In [65]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter = 1000)

from sklearn.model_selection import GridSearchCV
param_grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}

grid = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1, scoring='roc_auc')
grid.fit(X_train,y_train)

print("best parameters ",grid.best_params_)
print("roc_auc :",grid.best_score_)

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix
clf = LogisticRegression(max_iter = 1000,C= 2, penalty = 'l2', n_jobs=-1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_score= clf.predict_proba(X_test)
#y_score = clf.decision_function(X_test)
roc_auc =  roc_auc_score(y_test, y_score[:,1])

print('roc_auc: ', roc_auc)
print('CM: ', confusion_matrix(y_test, y_pred))

roc_auc:  0.7587939882020338
CM:  [[36986    97]
 [ 2809    78]]


In [ ]:
from sklearn.svm import SVC

clf = SVC() 

from sklearn.model_selection import GridSearchCV

param_grid = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10]}   ##kernel =  linear, poly, rbf

grid = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1, scoring = 'roc_auc')
grid.fit(X_train,y_train)

print("best parameters ",grid.best_params_)
print("roc_auc :",grid.best_score_)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, confusion_matrix
clf = SVC().fit(X_train, y_train)

y_pred = clf.predict(X_test)
    
roc_auc =  roc_auc_score(y_test, y_pred)
print('roc_auc: ', roc_auc)
print('CM: ', confusion_matrix(y_test, y_pred))

In [67]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

param_grid = {'splitter':('best','random')
              ,'criterion':("gini","entropy")
              ,"max_depth":[*range(1,10)]
              }

grid = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1, scoring = 'roc_auc')

grid.fit(X_train,y_train)

print("best parameters ",grid.best_params_)
print("roc_auc :",grid.best_score_)

best parameters  {'criterion': 'entropy', 'max_depth': 6, 'splitter': 'best'}
roc_auc : 0.7807551209588176


In [68]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
clf = DecisionTreeClassifier(criterion='entropy', max_depth= 8, splitter= 'best').fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
y_score= clf.predict_proba(X_test)
#y_score = clf.decision_function(X_test)
roc_auc =  roc_auc_score(y_test, y_score[:,1])


print('roc_auc: ', roc_auc)
print('CM: ', confusion_matrix(y_test, y_pred))

roc_auc:  0.786375512907083
CM:  [[37015    68]
 [ 2526   361]]


In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

param_grid = {'var_smoothing': [1e-2, 1e-3, 1e-4]}

grid = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1, scoring = 'roc_auc')
grid.fit(X_train,y_train)

print("best parameters ",grid.best_params_)
print("roc_auc :",grid.best_score_)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, confusion_matrix
clf = GaussianNB().fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
    
y_score= clf.predict_proba(X_test)
#y_score = clf.decision_function(X_test)
roc_auc =  roc_auc_score(y_test, y_score[:,1])

print('roc_auc: ', roc_auc)
print('CM: ', confusion_matrix(y_test, y_pred))

In [69]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clf = RandomForestClassifier(random_state = 0)

from sklearn.model_selection import GridSearchCV

param_grid =    { 'n_estimators': [200, 500],
                 'max_features': ['auto', 'sqrt', 'log2'],
                 'max_depth' : [4,5,6,7,8],
                 'criterion' :['gini', 'entropy']}
grid = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1, scoring = 'roc_auc')

grid.fit(X_train,y_train)

print("best parameters ",grid.best_params_)
print("roc_auc :",grid.best_score_)

best parameters  {'criterion': 'entropy', 'max_depth': 8, 'max_features': 'log2', 'n_estimators': 500}
roc_auc : 0.7893617968014635


In [71]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix

clf = RandomForestClassifier(n_estimators=500, max_features= 'log2', max_depth = 8, criterion = 'entropy' ).fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
y_score= clf.predict_proba(X_test)
#y_score = clf.decision_function(X_test)
roc_auc =  roc_auc_score(y_test, y_score[:,1])

print('roc_auc: ', roc_auc)
print('CM: ', confusion_matrix(y_test, y_pred))

roc_auc:  0.7973211657564689
CM:  [[37047    36]
 [ 2508   379]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state = 0)

from sklearn.model_selection import GridSearchCV

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.7,  0.8,  0.9,  1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1, scoring = 'roc_auc')

grid.fit(X_train,y_train)

print("best parameters ",grid.best_params_)
print("roc_auc :",grid.best_score_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve

clf = GradientBoostingClassifier(random_state = 0).fit(X_train, y_train)


y_pred = clf.predict(X_test)
y_score= clf.predict_proba(X_test)
#y_score = clf.decision_function(X_test)
roc_auc =  roc_auc_score(y_test, y_score[:,1])

print('roc_auc: ', roc_auc)
print('CM: ', confusion_matrix(y_test, y_pred))


In [ ]:
list(test)

In [19]:
test.drop([ 'agency_name',
 'inspector_name',
 'violator_name',
 'violation_street_number',
 'violation_street_name',
 'violation_zip_code',
 'mailing_address_str_number',
 'mailing_address_str_name',
 'city',
 'state',
 'zip_code',
 'non_us_str_code',
 'country',
 'ticket_issued_date',
 'hearing_date',
 'violation_code',
 'violation_description',
 'disposition',
 'grafitti_status'], axis=1, inplace=True)
test

,ticket_id,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount
0,284932,200.0,20.0,10.0,20.0,0.0,0.0,250.0
1,285362,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0
2,285361,100.0,20.0,10.0,10.0,0.0,0.0,140.0
3,285338,200.0,20.0,10.0,20.0,0.0,0.0,250.0
4,285346,100.0,20.0,10.0,10.0,0.0,0.0,140.0
...,...,...,...,...,...,...,...,...
60996,376496,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0
60997,376497,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0
60998,376499,100.0,20.0,10.0,10.0,0.0,0.0,140.0
60999,376500,100.0,20.0,10.0,10.0,0.0,0.0,140.0


In [27]:
x = clf.predict_proba(test)[:,1]

In [57]:
y = pd.Series(x, index = test['ticket_id'])
y

ticket_id
284932    0.00
285362    0.00
285361    0.00
285338    0.06
285346    0.00
          ... 
376496    0.00
376497    0.00
376499    0.58
376500    0.58
369851    0.70
Length: 61001, dtype: float64

In [59]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    train = pd.read_csv('train.csv', encoding='cp1252',  low_memory=False, ) 
    test = pd.read_csv('test.csv')
    train = train.dropna(subset=['compliance'])
    #train = train[np.isfinite(train['compliance'])]

    X_train = train.iloc[:,:-1]
    y_train = train['compliance']
    X_train.drop([ 'agency_name',
     'inspector_name',
     'violator_name',
     'violation_street_number',
     'violation_street_name',
     'violation_zip_code',
     'mailing_address_str_number',
     'mailing_address_str_name',
     'city',
     'state',
     'zip_code',
     'non_us_str_code',
     'country',
     'ticket_issued_date',
     'hearing_date',
     'violation_code',
     'violation_description',
     'disposition',
     'payment_date',
     'payment_status',
     'collection_status',
     'grafitti_status',
     'compliance_detail',
     'balance_due',
    'payment_amount' ], axis=1, inplace=True)
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.metrics import roc_auc_score, confusion_matrix

    clf = RandomForestClassifier(n_estimators=500, max_features= 'log2', max_depth = 8, criterion = 'entropy' ).fit(X_train, y_train)
      
    test.drop([ 'agency_name',
     'inspector_name',
     'violator_name',
     'violation_street_number',
     'violation_street_name',
     'violation_zip_code',
     'mailing_address_str_number',
     'mailing_address_str_name',
     'city',
     'state',
     'zip_code',
     'non_us_str_code',
     'country',
     'ticket_issued_date',
     'hearing_date',
     'violation_code',
     'violation_description',
     'disposition',
     'grafitti_status'], axis=1, inplace=True)
    x = clf.predict_proba(test)[:,1]
    y = pd.Series(x, index = test['ticket_id'])

    return y

In [60]:
blight_model()

ticket_id
284932    0.00
285362    0.00
285361    0.18
285338    0.03
285346    0.19
          ... 
376496    0.00
376497    0.00
376499    0.56
376500    0.56
369851    0.73
Length: 61001, dtype: float64

In [61]:
import numpy as np
bm = blight_model()
res = '{:40s}'.format('Object Type:')
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += '{:40s}'.format('Data Shape:')
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += '{:40s}'.format('Data Values Type:')
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += '{:40s}'.format('Data Values Infinity:')
res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
res += '{:40s}'.format('Data Values NaN:')
res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
res += '{:40s}'.format('Data Values in [0,1] Range:')
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += '{:40s}'.format('Data Values not all 0 or 1:')
res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
res += '{:40s}'.format('Index Type:')
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += '{:40s}'.format('Index Values:')
if bm.index.shape==(61001,):
    res +=['Failed: index values should match test.csv\n','Passed\n'
          ][all(pd.read_csv('test.csv',usecols=[0],index_col=0
                           ).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed: bm.index length should be 61001'
res += '{:40s}'.format('Can run model twice:')
bm2 = None
try:
    bm2 = blight_model()
    res += 'Passed\n'
except:
    res += ['Failed: second run of blight_model() threw an Exception']
res += '{:40s}'.format('Can run model twice with same results:')
if not bm2 is None:
    res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
        all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
print(res)

Object Type:                            Passed
Data Shape:                             Passed
Data Values Type:                       Passed
Data Values Infinity:                   Passed
Data Values NaN:                        Passed
Data Values in [0,1] Range:             Passed
Data Values not all 0 or 1:             Passed
Index Type:                             Passed
Index Values:                           Passed
Can run model twice:                    Passed
Can run model twice with same results:  Failed: second run of blight_model() produced different results (this might not be a problem)

